In [1]:
### importing the dependencies

In [2]:
import nltk
import numpy 
import json
import tensorflow
import random
import pickle

In [3]:
## reading the json file

with open("C:/Users/Tejaswini/Documents/POCS/chatbot(dl)/Intents.json") as file:
    data=json.load(file)

In [4]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [5]:
## to convert words into their base word ,we uses LancasterStemmer 

from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

### Converting the data into sentences tokenize and word tokenize

In [6]:
try:
    with open('data.pickle' , 'rb') as f:
        words,labels,training,output=pickle.load(f)
except:
    words=[] ## pattern sentences is tokenize
    labels=[] ## tag in intents
    docs_x=[] ## pattern in intents
    docs_y=[] ## tag in intents

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds=nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
        
        
        if intent["tag"] not in labels:
            labels.append(intent["tag"])
    
    ## Text Preprocessing
    ## words sorted in alphabetical and set gives only unique words ,then added these words into list
    words=[stemmer.stem(w.lower()) for w in words if w not in '?']
    words=sorted(list(set(words)))
    
    ## sort a/c to alphabetical orders
    labels=sorted(labels)
    
    training=[]
    output=[]

    ## it returns empty list of len(labels)
    out_empty=[0 for _ in range(len(labels))]
    
    ## it converts the words  text data in pattern to bag of words vector
    for x,doc in enumerate(docs_x):
        bag=[]
    
        wrds=[stemmer.stem(w) for w in doc]
    
        for w in words:     
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
            
        output_row=out_empty[:]
        output_row[labels.index(docs_y[x])]=1
    
        training.append(bag) ##  converted bag append to training
        output.append(output_row) ## it converts the labels text into bag of words vector
        
    
    ## training data 
    training=numpy.array(training)
    output=numpy.array(output) 
    
    with open('data.pickle' , 'wb') as f:
        pickle.dump((words,labels,training,output),f)

## Model Creation

In [7]:
### Dependencies for model creation

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout , Activation, Flatten , Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from keras.models import load_model

In [8]:
## model building 

model=Sequential()
model.add(Dense(128,input_shape=(len(training[0]),),activation='relu'))
model.add(Dropout(.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(.5))
model.add(Dense(len(output[0]),activation='softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               10880     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 14)                910       
                                                                 
Total params: 20,046
Trainable params: 20,046
Non-trainable params: 0
_________________________________________________________________


In [10]:
## compile and fit our model to find the accuracy

sgd_optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd_optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

C:\Users\Tejaswini\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [11]:
# fit the model

try:
    ## saving our model
    model.load('chatbot_Application_model')
    
except:
    model.fit(training, output, epochs=200, batch_size=5, verbose=1)
    model.save('chatbot_Application_model.h5')

Epoch 1/200
10/10 [==============================] - 0s 1ms/step - loss: 2.7165 - accuracy: 0.0000e+00
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.5390 - accuracy: 0.1064
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 2.4395 - accuracy: 0.1489
Epoch 4/200
10/10 [==============================] - 0s 1ms/step - loss: 2.2632 - accuracy: 0.2553
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1232 - accuracy: 0.2766
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 1.9619 - accuracy: 0.3617
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 1.8220 - accuracy: 0.4255
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6203 - accuracy: 0.5532
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4549 - accuracy: 0.5745
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3945 - accuracy: 0.5532
Epoch

Epoch 83/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0579 - accuracy: 0.9787
Epoch 84/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0137 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0689 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0189 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0377 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0255 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0437 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 4ms/step - loss: 0.1428 - accuracy: 0.9362
Epoch 92/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0906 - accuracy: 0.9787


10/10 [==============================] - 0s 4ms/step - loss: 0.0338 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0105 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0277 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0677 - accuracy: 0.9787
Epoch 170/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0290 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0279 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0131 - accuracy: 1.0000
Epoc

## To get predication of sentences and responses from our model

In [12]:
## To run our model we have to provide the input in the same way as we have done while creating our model. 
## So for this we have created a function which will perform text operations and then predict the label.


def bag_of_words(sen,words):
    bag=[0 for _ in range(len(words))]
    
    sen_words=nltk.word_tokenize(sen)
    sen_words=[stemmer.stem(word.lower()) for word in sen_words if not  '?']
    
    for s in sen_words:
        for i,w in enumerate(words):
            if w==s :
                bag[i]=1
    return numpy.array(bag)

In [13]:
def chat():
    print("Start talking with bot! (type quit to stop)")
    while True:
        inp=input('User : ')
        if inp.lower()=='quit':
            break
            
        result = model.predict([bag_of_words(inp,words)])
        result_index=numpy.argmax(result)
        tag=labels[result_index]
        
        
        
        for tg in data['intents']:
            if tg['tag']==tag:
                responses=tg['responses']
                
        print(random.choice(responses)) 

In [14]:
chat()

Start talking with bot! (type quit to stop)
User : quit
